In [11]:
from cartoframes.auth import (
    Credentials,
    set_default_credentials
)
from cartoframes.data.clients import DataObsClient
from cartoframes.viz import Layer


credentials = Credentials.from_file()
set_default_credentials(credentials)

In [12]:
do = DataObsClient(credentials)

### Spain provinces boundaries map

In [3]:
es_boundaries = do.boundaries(region='Spain')
es_boundaries.dataframe[['geom_name', 'geom_id']]

,geom_name,geom_id
0,Autonomous Community,es.cnig.ccaa
1,Municipality,es.cnig.muni
2,Province,es.cnig.prov
3,Sección Censal,es.ine.the_geom


In [4]:
es_provinces = do.boundaries(boundary='es.cnig.prov')
es_provinces.dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 2 columns):
the_geom     52 non-null object
geom_refs    52 non-null object
dtypes: object(2)
memory usage: 960.0+ bytes


In [ ]:
Layer(es_provinces.dataframe)

### Exploring Australia boundaries

In [13]:
au_boundaries = do.boundaries(region='Australia')
au_boundaries.dataframe[['geom_name', 'geom_id']]

,geom_name,geom_id
0,Commonwealth Electoral Divisions,au.geo.CED
1,Greater Capital City Statistical Areas,au.geo.GCCSA
2,Local Government Areas,au.geo.LGA
3,Postal Areas,au.geo.POA
4,Remoteness Areas,au.geo.RA
5,Statistical Area Level 1,au.geo.SA1
6,Statistical Area Level 2,au.geo.SA2
7,Statistical Area Level 3,au.geo.SA3
8,Statistical Area Level 4,au.geo.SA4
9,State Electoral Divisions,au.geo.SED


In [14]:
au_postal_areas = do.boundaries(boundary='au.geo.POA')
# Be careful with the dataframe size here before try to render a Layer with it:
au_postal_areas.dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2513 entries, 0 to 2512
Data columns (total 2 columns):
the_geom     2513 non-null object
geom_refs    2513 non-null object
dtypes: object(2)
memory usage: 39.4+ KB


### Looking for "median income" data in US area

In [9]:
tracts = do.boundaries(
                    boundary='us.census.tiger.census_tract',
                    region=[-112.096642,43.429932,-111.974213,43.553539]
)

In [ ]:
tracts.upload(table_name='idaho_falls_tracts', credentials=credentials, if_exists='replace')

In [10]:
median_income_meta = do.discovery(
                    'idaho_falls_tracts',
                    keywords='median income')
median_income_meta

,denom_aggregate,denom_colname,denom_description,denom_geomref_colname,denom_id,denom_name,denom_reltype,denom_t_description,denom_tablename,denom_type,...,numer_timespan,numer_type,score,score_rank,score_rownum,suggested_name,target_area,target_geoms,timespan_rank,timespan_rownum
0,sum,households,A count of the number of households in each ge...,geoidsl,us.census.acs.B11001001,Households,universe,NaN,obs_b973a5f9a7faed002c70ca7df10c59b1e39c6654,Numeric,...,2006 - 2010,Numeric,36.521909,1.0,1.0,median_income_2006_2010,NaN,NaN,6.0,3.0
1,sum,households,A count of the number of households in each ge...,geoidsl,us.census.acs.B11001001,Households,universe,NaN,obs_b393b5b88c6adda634b2071a8005b03c551b609a,Numeric,...,2010 - 2014,Numeric,36.521909,1.0,1.0,median_income_2010_2014,NaN,NaN,4.0,2.0
2,sum,households,A count of the number of households in each ge...,geoidsl,us.census.acs.B11001001,Households,universe,NaN,obs_9effa577f8b5d674d620dfbaf4aec55e70ac06b8,Numeric,...,2011 - 2015,Numeric,36.521909,1.0,1.0,median_income_2011_2015,NaN,NaN,3.0,1.0
3,sum,households,A count of the number of households in each ge...,geoidsl,us.census.acs.B11001001,Households,universe,NaN,obs_b973a5f9a7faed002c70ca7df10c59b1e39c6654,Numeric,...,2006 - 2010,Numeric,36.521909,1.0,1.0,median_income_2006_2010,NaN,NaN,6.0,3.0
4,sum,households,A count of the number of households in each ge...,geoidsl,us.census.acs.B11001001,Households,universe,NaN,obs_b393b5b88c6adda634b2071a8005b03c551b609a,Numeric,...,2010 - 2014,Numeric,36.521909,1.0,1.0,median_income_2010_2014,NaN,NaN,4.0,2.0
5,sum,households,A count of the number of households in each ge...,geoidsl,us.census.acs.B11001001,Households,universe,NaN,obs_9effa577f8b5d674d620dfbaf4aec55e70ac06b8,Numeric,...,2011 - 2015,Numeric,36.521909,1.0,1.0,median_income_2011_2015,NaN,NaN,3.0,1.0


In [ ]:
# Warning: right now the median_income_meta may have duplicates that breaks the following `augment` call
median_income_unique = median_income_meta.loc[:2]

In [ ]:
idaho_falls_income = do.augment(
                    'idaho_falls_tracts',
                    median_income_unique,
                    how='geom_refs')

In [ ]:
idaho_falls_income.dataframe.head(5)

In [ ]:
Layer(idaho_falls_income.dataframe)